In [1]:
import gammapy 
print(f'loaded gammapy version: {gammapy.__version__} ' )
print(f'Supposed to be 1.0 (21-12-2022)' )

loaded gammapy version: 0.1.dev19174+g56b5bdf 
Supposed to be 1.0 (21-12-2022)


In [2]:
#get_ipython().system('jupyter nbconvert --to script 1-Nui_Par_Fitting.ipynb')
import pyximport

pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import gammapy

# from gammapy.datasets import MapDataset
from gammapy.maps import Map
from astropy.coordinates import SkyCoord, Angle
from gammapy.modeling import Fit, Parameter, Parameters, Covariance
from gammapy.datasets import MapDataset #, MapDatasetNuisance
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    create_crab_spectral_model,
    SkyModel,
    PointSpatialModel,
    ShellSpatialModel,
    GeneralizedGaussianSpatialModel,
    TemplateSpatialModel,
    LogParabolaSpectralModel,
    GaussianSpatialModel,
    DiskSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SpatialModel,
    FoVBackgroundModel,
)
from gammapy.estimators import TSMapEstimator, ExcessMapEstimator
from gammapy.maps import MapAxis
from gammapy.modeling.models.spectral import scale_plot_flux
from gammapy.estimators import  FluxPointsEstimator
    
from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys

from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys
sys.path.append('/home/katrin/Documents/nuisance_summary/')
sys.path.append('../')
from Dataset_Creation import sys_dataset
import Dataset_load 

## Read dataset

In [3]:

scale  = 0.05
scaled_amplitude = Parameter('amplitude',
                             value = 4.11e-11 * scale)
dataset_asimov  = Dataset_load.create_asimov(model = 'pl',
                                            parameters = Parameters([scaled_amplitude]))

update evaluator


## Varying the Exposure

In [16]:
mask = dataset_asimov.mask.data.sum(axis=2).sum(axis=1)>0

ebins = dataset_asimov.counts.geom.axes[0].center[mask]
print(len(ebins))

16


In [20]:
shift = +0.1
tilt = 0.015


N = 495
sigma_a = shift
sigma_i = tilt

save = True
save_flux = True




In [ ]:
for n in range(N):
    print()
    print('====' * 30)
    print(n)
    print('====' * 30)
    
    shift_rnd = np.random.normal(0, shift, 1)
    tilt_rnd = np.random.normal(0, tilt, 1)
    print(f"shift:, {shift_rnd}, tilt: {tilt_rnd}" )
    sys_d_cor = sys_dataset(dataset_asimov= dataset_asimov,
                    shift = shift_rnd, 
                    tilt = tilt_rnd,
                    rnd = True)
    dataset = sys_d_cor.create_dataset()
    fit_cor = Fit(store_trace=False)
    minuit_opts = {"tol": 0.001, "strategy": 2}
    fit_cor.optimize_opts = minuit_opts
    result_cor = fit_cor.run([dataset])


    if save:
        with open("data/7a_P_draw_info.txt", "a") as myfile:
            myfile.write(str(float(shift_rnd)) + '    '+ str(float(tilt_rnd)) + '    ' +  str(float(dataset.stat_sum())) + '\n')

    stri = ""
    for p in ['amplitude', 'index', 'norm', 'tilt']:
        stri += str(dataset.models.parameters[p].value)  + '   ' +  str(dataset.models.parameters[p].error)  + '   '
    print(stri)
    if save:
        with open("data/7a_P_draw_par.txt", "a") as myfile:
            myfile.write(stri + '\n')



    fluxes = []
    for e in ebins:
        flux =  dataset.models[0].spectral_model(e)
        fluxes.append(flux.value)

    ff = str()
    for f in fluxes:
        ff += str(f) + "  "
    #print(ff)
    if save:
        with open("data/7a_P_draw_flux.txt", "a") as myfile:
            myfile.write( ff + '\n')


    energy_bounds = (ebins[0], ebins[-1] ) * u.TeV

    energy_min, energy_max = energy_bounds
    energy = MapAxis.from_energy_bounds(
        energy_min,
        energy_max,
        16,
    )

    fluxe2, _ = dataset.models[0].spectral_model._get_plot_flux(sed_type='dnde', energy=energy)
    fluxe2 = scale_plot_flux(fluxe2, energy_power=2)
    fluxe2 = fluxe2.quantity[:, 0, 0]
    fluxe2 = np.array(fluxe2)   
    ff = str()
    for f in fluxe2:
        ff += str(f) + "  "
    if save:
        with open("data/7a_P_draw_flux2e.txt", "a") as myfile:
            myfile.write( ff + '\n')

    energy_edges = dataset.geoms['geom'].axes[0].edges
    esti  = FluxPointsEstimator(energy_edges= energy_edges)
    fluxpoints = esti.run([dataset])
    if save_flux:
        fluxpoints.write(f'data/fluxpoints/6_fluxpoints_{shift_rnd[0]:.6}.fits')
    #except:
    #    pass



0
shift:, [0.05411072], tilt: [0.02937959]
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
update evaluator
1.8929272134274925e-12   8.940224019988683e-14   2.916986789884011   0.06103709108713398   1.0225085560799279   0.018560429121934303   0.005355315887017122   0.015444648387057178   

0.10000000000000002 TeV 0.1333521432163324 TeV
norm 1.0 0
update evaluator

0.1333521432163324 TeV 0.1778279410038923 TeV
norm 1.0 0
update evaluator

0.1778279410038923 TeV 0.23713737056616552 TeV
norm 1.0 0
update evaluator

0.23713737056616552 TeV 0.316227766016838 TeV
norm 1.0 0
update evaluator

0.316227766016838 TeV 0.4216965034285822 TeV
norm 1.0 0
update evaluator

0.4216965034285822 TeV 0.5623413251903491 TeV
norm 1.0 0
update evaluator

0.5623413251903491 TeV 0.7498942093324559 TeV
norm 1.0 0
update evaluator

0.7498942093324559 TeV 1.0 TeV
norm 1.0 0
u

Position <SkyCoord (ICRS): (ra, dec) in deg
    (40.20892478, -16.45131839)> is outside valid IRF map range, using nearest IRF defined within


update evaluator
2.1284370911747354e-12   nan   2.794387664193396   nan   1.191256415666238   nan   0.025284495518188576   nan   

0.10000000000000002 TeV 0.1333521432163324 TeV
norm 1.0 0
update evaluator

0.1333521432163324 TeV 0.1778279410038923 TeV
norm 1.0 0
update evaluator

0.1778279410038923 TeV 0.23713737056616552 TeV
norm 1.0 0
update evaluator

0.23713737056616552 TeV 0.316227766016838 TeV
norm 1.0 0
update evaluator

0.316227766016838 TeV 0.4216965034285822 TeV
norm 1.0 0
update evaluator

0.4216965034285822 TeV 0.5623413251903491 TeV
norm 1.0 0
update evaluator

0.5623413251903491 TeV 0.7498942093324559 TeV
norm 1.0 0
update evaluator

0.7498942093324559 TeV 1.0 TeV
norm 1.0 0
update evaluator

1.0 TeV 1.333521432163324 TeV
update evaluator
norm 0.9595791345847915 0.05706234556409874

1.333521432163324 TeV 1.7782794100389232 TeV
update evaluator
norm 0.9499697927692258 0.06872130501163728

1.7782794100389232 TeV 2.371373705661655 TeV
update evaluator
norm 0.935718359967685